# Data Cleaning for Our Dataset
### Evan Schubert
### CSC 2621-141
### Created on: 04/10/2025

What Each Column Represents
Transaction_ID - arbitrary unique ID given to each transaction<br>
User_ID - The unique identifier for each user<br>
Transaction Amount - amount of money transacted<br>
Transaction Type - ATM withdrawal, Bill Payment, POS payment, bank transfer, online purchase<br>
Time of Transaction - Military time for the time the transaction occured, 0 through 23<br>
Device Used - Tablet, mobile, desktop, unknown device<br>
Location - San Francisco, New York, Chicago, Boston, Houston, Miami, Los Angeles, Seattle<br>
Previous Fraudulent Transactions - How many previous fraudelent transactions does this user have, 0 through 4<br>
Account Age - How long the account has been active in possibly months, 0 through 119<br>
Number of Transactions last 24H - The number of transactions in the last 24 hours for this user, 1 through 14<br>
Payment Method - Debit card, credit card, UPI, net banking, invalid method<br>
Fraudulent - boolean representing whether the transaction was fraudulent.<br>

In [33]:
import pandas as pd
from scipy import stats
import numpy as np

fraud_df = pd.read_csv("Fraud Detection Dataset.csv")

In [35]:
fraud_df.dropna(inplace = True, subset = ['Device_Used', 'Location', 'Payment_Method'])

In [37]:
fraud_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43869 entries, 0 to 50999
Data columns (total 12 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Transaction_ID                    43869 non-null  object 
 1   User_ID                           43869 non-null  int64  
 2   Transaction_Amount                41687 non-null  float64
 3   Transaction_Type                  43869 non-null  object 
 4   Time_of_Transaction               41654 non-null  float64
 5   Device_Used                       43869 non-null  object 
 6   Location                          43869 non-null  object 
 7   Previous_Fraudulent_Transactions  43869 non-null  int64  
 8   Account_Age                       43869 non-null  int64  
 9   Number_of_Transactions_Last_24H   43869 non-null  int64  
 10  Payment_Method                    43869 non-null  object 
 11  Fraudulent                        43869 non-null  int64  
dtypes: float6

In [10]:
fraud_df.head()

,Transaction_ID,User_ID,Transaction_Amount,Transaction_Type,Time_of_Transaction,Device_Used,Location,Previous_Fraudulent_Transactions,Account_Age,Number_of_Transactions_Last_24H,Payment_Method,Fraudulent
0,T1,4174,1292.76,ATM Withdrawal,16.0,Tablet,San Francisco,0,119,13,Debit Card,0
1,T2,4507,1554.58,ATM Withdrawal,13.0,Mobile,New York,4,79,3,Credit Card,0
3,T4,2294,100.10,Bill Payment,15.0,Desktop,Chicago,4,3,4,UPI,0
4,T5,2130,1490.50,POS Payment,19.0,Mobile,San Francisco,2,57,7,Credit Card,0
5,T6,2095,2372.04,ATM Withdrawal,15.0,Desktop,Boston,3,96,14,Credit Card,0


In [12]:
print(fraud_df.columns)

Index(['Transaction_ID', 'User_ID', 'Transaction_Amount', 'Transaction_Type',
       'Time_of_Transaction', 'Device_Used', 'Location',
       'Previous_Fraudulent_Transactions', 'Account_Age',
       'Number_of_Transactions_Last_24H', 'Payment_Method', 'Fraudulent'],
      dtype='object')


In [39]:
fraud_df = fraud_df.astype({ 'Transaction_Type': 'category', 'Device_Used': 'category', 'Location': 'category','Payment_Method': 'category'})

## Filling in missing values

In [49]:
fraud_df["Transaction_Amount_fill_mean"] = fraud_df["Transaction_Amount"]
fraud_df["Transaction_Amount_fill_median"] = fraud_df["Transaction_Amount"]
fraud_df["Transaction_Amount_fill_mode"] = fraud_df["Transaction_Amount"]

values = {"Transaction_Amount_fill_mean": fraud_df["Transaction_Amount"].mean(), "Transaction_Amount_fill_median": fraud_df["Transaction_Amount"].median(), 
         "Transaction_Amount_fill_mode": stats.mode(fraud_df["Transaction_Amount"], nan_policy="omit").mode}
fraud_df.fillna(value = values, inplace = True)

In [53]:
fraud_df.head(10)

,Transaction_ID,User_ID,Transaction_Amount,Transaction_Type,Time_of_Transaction,Device_Used,Location,Previous_Fraudulent_Transactions,Account_Age,Number_of_Transactions_Last_24H,Payment_Method,Fraudulent,Transaction_Amount_fill_mean,Transaction_Amount_fill_median,Transaction_Amount_fill_mode
0,T1,4174,1292.76,ATM Withdrawal,16.0,Tablet,San Francisco,0,119,13,Debit Card,0,1292.760000,1292.76,1292.76
1,T2,4507,1554.58,ATM Withdrawal,13.0,Mobile,New York,4,79,3,Credit Card,0,1554.580000,1554.58,1554.58
3,T4,2294,100.10,Bill Payment,15.0,Desktop,Chicago,4,3,4,UPI,0,100.100000,100.10,100.10
4,T5,2130,1490.50,POS Payment,19.0,Mobile,San Francisco,2,57,7,Credit Card,0,1490.500000,1490.50,1490.50
5,T6,2095,2372.04,ATM Withdrawal,15.0,Desktop,Boston,3,96,14,Credit Card,0,2372.040000,2372.04,2372.04
6,T7,4772,544.81,Bill Payment,2.0,Tablet,Boston,3,6,9,UPI,1,544.810000,544.81,544.81
7,T8,4092,635.75,ATM Withdrawal,13.0,Tablet,Boston,2,13,10,Debit Card,0,635.750000,635.75,635.75
8,T9,2638,2318.87,Bank Transfer,NaN,Mobile,San Francisco,4,110,12,Debit Card,0,2318.870000,2318.87,2318.87
9,T10,3169,3656.17,Bill Payment,3.0,Mobile,Chicago,4,66,3,Net Banking,0,3656.170000,3656.17,3656.17
10,T11,1466,NaN,Online Purchase,3.0,Tablet,Houston,1,4,6,Net Banking,0,2993.984644,2522.53,49997.80


In [43]:
df["Transaction_Amount_fill_KNN"] = df["Transaction_Amount"]
df_drop_na = titanic_data
df_drop_na = df_drop_na.dropna(subset = "Transaction_Amount")

#x_train = df_drop_na[["SibSp", "Parch", "Fare"]]
y_train = df_drop_na["Transaction_Amount"]
#x_test = df_drop_na[["SibSp", "Parch", "Fare"]]
model = KNeighborsRegressor(n_neighbors=5)
model.fit(x_train, y_train)

answers = model.predict(x_test)
index = 0
for amount in df["Age_fill_knn"]:
    if(np.isnan(amount)):
        df.at[index, "Age_fill_knn"] = answers[index]
    index += 1

NameError: name 'titanic_data' is not defined